# Step 12
in this step we prepare data for Deep Learning Model
1) add regions feature

and in Step 13:
 <br>
2) convert TTC feature to a vector only with TTC values


### Load data from folder: "10_data_with_ttc_and_distance_2_zebra"

In [1]:
from Utils import *

In [2]:
root_dir = r"C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation"

In [3]:
directory = r'C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\10_data_with_ttc_and_distance_2_zebra'
# Get a list of files in the directory
file_name_list = os.listdir(directory)
file_name_list

['18_data_with_ttc_and_distance_2_zebra.csv',
 '19_data_with_ttc_and_distance_2_zebra.csv',
 '20_data_with_ttc_and_distance_2_zebra.csv',
 '21_data_with_ttc_and_distance_2_zebra.csv',
 '22_data_with_ttc_and_distance_2_zebra.csv',
 '23_data_with_ttc_and_distance_2_zebra.csv',
 '24_data_with_ttc_and_distance_2_zebra.csv',
 '25_data_with_ttc_and_distance_2_zebra.csv',
 '26_data_with_ttc_and_distance_2_zebra.csv',
 '27_data_with_ttc_and_distance_2_zebra.csv',
 '28_data_with_ttc_and_distance_2_zebra.csv',
 '29_data_with_ttc_and_distance_2_zebra.csv']

In [4]:
recordingId_list=get_recordingsId_list(file_name_list)
recordingId_list

['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']

In [5]:
dfs = load_dfs(file_name_list, root_dir, directory)

In [6]:
folder_name="12_data_regions_feature"
create_folder_to_save_data(root_dir, folder_name)

Folder: '12_data_regions_feature' already exists


## Create regions ditance features

we now add for each region a feature of distance.

we have 4 regions: zebra, 1, 2, and 3

In [7]:
# zone 1
xy_1 =  [48, -37]        # center [m]
r1=     4.89            # [m]
# zone 2
xy_2 =  [43, -17]       # center [m]
r2=     4.89            # [m]
# zone 3
xy_3 =  [38, -31.5]     # center [m]
r3=     4.89            # [m]
# zone zebra
xy_zebra = [61.5, -28.1]    # center [m]
r_zebra= 4.89               # [m]

calculate for each pedestrian using "normalized_distance_from_center" the distance from each region and add new columns.

In [8]:
df=dfs[0].copy()

In [9]:
# create and initialize empty columns for zones feature
df["zone_zebra"] = np.nan
df["zone_1"] = np.nan
df["zone_2"] = np.nan
df["zone_3"] = np.nan

In [10]:
def create_zones_feature(df, xy_zebra, r_zebra, xy_1, r1, xy_2, r2, xy_3, r3):
    """
    Create zones features for the given DataFrame.
    """
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # take the coordinates of position of object (from all classes)
        point_xy= row[["xCenter", "yCenter"]].values
        # zone_zebra
        distance_zone_zebra=normalized_distance_from_center(point_xy, xy_zebra, r_zebra)
        df.at[index, "zone_zebra"] = distance_zone_zebra
        # zone_1
        distance_zone_1=normalized_distance_from_center(point_xy, xy_1, r1)
        df.at[index, "zone_1"] = distance_zone_1
        # zone_2
        distance_zone_2=normalized_distance_from_center(point_xy, xy_2, r2)
        df.at[index, "zone_2"] = distance_zone_2
        # zone_3
        distance_zone_3=normalized_distance_from_center(point_xy, xy_3, r3)
        df.at[index, "zone_3"] = distance_zone_3
    return df

In [11]:
for x in tqdm(range(len(dfs))):
    dfs[x] = create_zones_feature(dfs[x], xy_zebra, r_zebra, xy_1, r1, xy_2, r2, xy_3, r3)
print('zones features created')

100%|██████████| 12/12 [02:00<00:00, 10.08s/it]

zones features created


remove some unnecessary features:

In [12]:
columns_to_drop = ['hx', 'hy', 'width', 'length', 'distance2zebra']

In [13]:
for x in range(len(dfs)):
    dfs[x].drop(columns=columns_to_drop, inplace=True)

In [14]:
dfs[3].describe()

,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,xVelocity,yVelocity,num_of_pairs,zone_zebra,zone_1,zone_2,zone_3
count,54919.0,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000,54919.000000
mean,21.0,284.452357,3140.672991,70.206449,50.526021,-28.245959,145.632227,0.609219,-0.018308,1.188714,0.097125,0.036053,0.041224,0.059633
std,0.0,158.591036,1915.231800,61.642802,19.032777,8.357139,120.211367,3.156981,1.228876,1.827647,0.296130,0.186424,0.198811,0.236808
min,21.0,0.000000,0.000000,0.000000,7.271160,-56.570290,0.010390,-11.708960,-6.722730,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21.0,140.000000,1358.000000,25.000000,37.195025,-34.821925,13.535815,-1.069150,-0.780945,0.000000,0.000000,0.000000,0.000000,0.000000
50%,21.0,288.000000,3131.000000,53.000000,51.694820,-28.323690,177.059170,0.744120,0.002230,0.000000,0.000000,0.000000,0.000000,0.000000
75%,21.0,425.000000,4910.000000,97.000000,63.662515,-22.325730,241.582865,1.609125,0.652455,2.000000,0.000000,0.000000,0.000000,0.000000
max,21.0,549.000000,6221.000000,469.000000,93.163510,-4.190870,359.998400,13.721860,7.012830,8.000000,1.000000,1.000000,1.000000,1.000000


## save to csv

In [16]:
file_name_data_list = save_dfs(root_dir, dfs, recordingId_list, folder_name, '_data_with_zones.csv')

Saving CSV files: 100%|██████████| 12/12 [00:03<00:00,  3.45it/s]

files saved to: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\12_data_regions_feature
list of files saved:
['18_data_with_zones.csv', '19_data_with_zones.csv', '20_data_with_zones.csv', '21_data_with_zones.csv', '22_data_with_zones.csv', '23_data_with_zones.csv', '24_data_with_zones.csv', '25_data_with_zones.csv', '26_data_with_zones.csv', '27_data_with_zones.csv', '28_data_with_zones.csv', '29_data_with_zones.csv']
